In [1]:
import os
from py2neo import Graph
from pyhanlp import *
from sklearn.naive_bayes import MultinomialNB

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

为movie.csv,person.csv添加自定义词典

  对于带空格的自定义词无法在词库文件中添加，
  因为词库文件中一个词条的格式为“word pos   frequency”, 
  即“词 词性 词频”，
  举例：“单身狗 n 1024”，其中是以空格作为分隔符，所以如果自定义词中带空格，会引起格式错误。

In [3]:
CustomDictionary = JClass("com.hankcs.hanlp.dictionary.CustomDictionary")
with open("./question/movie.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        CustomDictionary.add(line, "nm 1024")

In [4]:
with open("./question/person.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        CustomDictionary.add(line,"nr 1024")

In [5]:
# 其实没用到，顺便添加
with open("./question/rating.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        CustomDictionary.add(line,"x 1024")

In [6]:
# 其实没用到，顺便添加
with open("./question/genre.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        CustomDictionary.add(line,"ng 1024")

In [7]:
# 因为和data/custom中的现代汉语词典中 卧虎藏龙这个成语冲突，所以将txt中nz更改为nm

In [8]:
# CustomDictionary.add("卧虎藏龙", "nm 1024")
# for term in HanLP.segment("卧虎藏龙"):
#     print(term)

In [9]:
# 强制自定义词典分词

In [10]:
JClass("com.hankcs.hanlp.seg.Segment").enableCustomDictionaryForcing

<java method `enableCustomDictionaryForcing' of 'com.hankcs.hanlp.seg.Segment'>

In [11]:
abstractMap = {}
vocabulary = {}
questions_pattern = None

In [12]:
# 加载词典，构建词典的代码见build_vocab
def load_vocabulary():
    with open("./question/vocabulary.txt", "r") as f:
        data = f.readlines()
    vocabulary = {}
    for line in data:
        tokens = line.split(":")
        index = int(tokens[0])
        word = tokens[1].strip()
        vocabulary[word] = index
    return vocabulary

In [13]:
# pyhanlp分词 得到sentence的独热向量
def sentence_to_arrays(sentence):
    global vocabulary
    # vector是one_hot向量表示，初始化为0，vector是51维 
    # vocabulary {'演员': 0,'演': 1,'了': 2,...}
    vector = [0] * len(vocabulary.keys())
    # pyhanlp分词后结果
    for term in HanLP.segment(sentence):
        if term.word in vocabulary:
            # get得到key值的value(index)序号
            index = vocabulary.get(term.word)
            vector[index] = 1
    # 返回sentence的51维的独热向量vector
    return vector

In [14]:
def query_abstract(query_sentence):
    """将HanLp分词后的关键word，用抽象词性xx替换

    :param query_sentence: 查询句子
    :return:
    """
    # 句子抽象化
    terms = HanLP.segment(query_sentence)
    abstract_query = ""
    global abstractMap
    nr_count = 0
    for term in terms:
        print(term)
        
        if "nm" in str(term):  # nm 电影名
            abstract_query += "nm "
            abstractMap["nm"] = term.word
        elif "nr" in str(term) and nr_count == 0:  # nr 人名
            abstract_query += "nr "
            abstractMap["nr"] = term.word
            nr_count += 1
        elif "nr" in str(term) and nr_count == 1:  # nr 人名 再出现一次，改成ntr
            abstract_query += "ntr "
            abstractMap["ntr"] = term.word
            nr_count += 1
        ### 其实这两个没用到，因为问题模板只有8个。
        elif "x" in str(term):  # x  评分
            abstract_query += "x "
            abstractMap["x"] = term.word
        elif "ng" in str(term):  # ng 类型
            abstract_query += "ng "
            abstractMap["ng"] = term.word
        else:
            abstract_query += term.word + " "
    print("========HanLP分词结束========")
    return abstract_query


In [15]:
# 问题分类
def query_classify(sentence):
    # terms 是51维独热向量。
    terms = sentence_to_arrays(sentence)
    # 朴素贝叶斯分类
    global clf
    index = int(clf.predict([terms])[0])
    print("the model index is " + str(index))
    # 返回对应的问题模板，和model_index
    return questions_pattern.get(index), index

In [16]:
# 将sentence还原成问题模板的语句。
def sentenceReduction(query_pattern):
    """
    将句子模板还原成正常的语句（分词关键word的抽象词性替换成原有的word）
    :param str_pattern:
    :return:
    """
    # 替换的模板是每个类的最后一个，当然这个可以更改！
    # abstractMap是字典，key是词性,value是单词
    global abstractMap
    for key in abstractMap.keys():
        if key in query_pattern:
            value = abstractMap.get(key)
            # 将value值替换为抽象map的值
            query_pattern = query_pattern.replace(key, value)
    
#     abstractMap = {}
    return query_pattern

In [17]:
def analysis(query_sentence):
    # 原始问句
    print("原始句子：" + query_sentence)
    print("========HanLP开始分词========")

    # 抽象句子，利用HanPL分词，将关键字进行词性抽象
    abstract_str = query_abstract(query_sentence)
    print("句子抽象化结果：" + abstract_str)

    # 将抽象的句子与模板进行匹配，拿到句子对应的模板
    str_pattern, model_index = query_classify(abstract_str)
    print("句子套用模板结果：" + str_pattern)

    # 模板还原成句子，此时问题已转换为我们熟悉的操作
    final_pattern = sentenceReduction(str_pattern)
    print("原始句子替换成系统可识别的结果：" + final_pattern)

    result = [model_index]
    result = result + final_pattern.split(" ")
    print("==========================")
#     print(result)
    return result

In [18]:
l_total=[]

In [30]:
def load_data(path="question"):
    X = []
    Y = []
    for root, dirs, files in os.walk(path):
        for f_name in files:
            # 对于其他文件不识别
            if "【" in f_name:  # 如：【3】剧情.txt
                # e.g. value是数字，代表第几个模板
                value = f_name.split("】")[0].replace("【", "")
                with open(os.path.join(root, f_name)) as f:
                    data = f.readlines()
                for sentence in data:
#                     l=[]
                    # 将每一个问题模板(共60个)转化成51维独热向量，append到X中
                    # y记录第几个模板
#                     terms = HanLP.segment(sentence)
#                     for term in terms:
#                         l.append(term.word)
#                     l_total.append(l)
#                     s =" ".join(l)
                    x = sentence_to_arrays(sentence.strip())
                    X.append(x)
                    Y.append(int(value))
    return X, Y


In [31]:
X,Y = load_data()

In [28]:
len(vocabulary)

0

In [80]:
corpus = open("./question/total_question.txt")

In [81]:
vocabulary = load_vocabulary()
#cv=TfidfVectorizer(binary=False,decode_error='ignore',stop_words='english')
cv=TfidfVectorizer(binary=False,decode_error='ignore')
# vec=cv.fit_transform(corpus.readlines())
vec=cv.fit_transform(l_total)
arr=vec.toarray() #文本特征值矩阵向量arr

AttributeError: 'list' object has no attribute 'lower'

In [70]:
arr[2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [19]:
def load_question_templates():
    path = "./question/question_classification.txt"
    with open(path, "r") as f:
        lines = f.readlines()
    questions_pattern = {}
    for line in lines:
        terms = line.split(":")
        questions_pattern[int(terms[0])] = terms[1].strip()
    # 返回问题模板的字典
    return questions_pattern

In [20]:
class QuestionService(object):

    def __init__(self):
        self.g = self.g = Graph(
            host="localhost",
            http_port=7474,
            user="neo4j",
            password="zzxxxx520"
        )

    def answer(self, question) -> str:
        # result [4, 'xx这部电影的评分是多少']
        result = analysis(question)
        answer = None
        model_index = result[0]

        if model_index == 4:
            answer = self.getMovieRating(result)
            if answer=="0":
                answer="评分为0，意味着暂无评分哦"
        elif model_index == 6:
            answer = self.getMovieReleaseDate(result)
        elif model_index == 3:
            answer = self.getMovieTypes(result)
        elif model_index == 2:
            answer = self.getMovieActors(result)
        elif model_index == 1:
            #  nm 电影作品 == 演员的电影作品有哪些
            actorMovies = self.getActorMovies(result)
            if actorMovies:
                answer = None
            else:
                answer = actorMovies
        elif model_index == 8:
            answer = self.getMoviesCount(result)
        elif model_index == 5:
            answer = self.getActorMoviesType(result)  
        elif model_index == 7:
            answer = self.getSameMovies(result)   
        #         elif model_index == 9:
#             # nm 简介 == 电影简介、详情
#             title = result[1]
#             answer = self.getMovieInfo(title)
        #         elif model_index == 5:
#             #  nnt 介绍 == 演员简介
#             answer = self.getActorInfo(result)
#         elif model_index == 8:
#             answer = self.getActorMoviesByHScore(result)
#         elif model_index == 9:
#             answer = self.getActorMoviesByLScore(result)
#         elif model_index == 11:
#             answer = self.getActorMovies(result)
#         elif model_index == 13:
#             # nnt  出生日期 == 演员出生日期 * /
#             answer = self.getActorBirth(result)
#         elif model_index == 5:
#             answer = self.getMoviesByType(result)s
        global abstractMap
        abstractMap = {}
        if answer:
            return answer
        else:
            return "sorry,小可爱,我没有找到你要的答案,换个提问方式试试哦,如果还是没有,可能没有录入这个数据哦"
        
    def getActorMovies(self, result):
        """1对应问题模板1 == nnt(演员) 电影作品

        :param name: 演员名
        :return:
        """
        global abstractMap
        try:
            name = abstractMap['nr']
        except:
            return None
        query = f"match(n:PersonItem)-[:Person2Movie]->(m:MovieItem) where n.name='"+ name +"' return m.title"
        answer = self.g.run(query).data()
        res=name
        res+="演过"
        for i in range(len(answer)-1):
            res+=answer[i]["m.title"]
            res+="、"
        res+=answer[len(answer)-1]["m.title"]
        res+="。"
        if answer:
            return res
        return None
    def getMovieActors(self, result):
        """2 对应问题模板2 == nm(电影) 演员

        :return: 返回电影的演员有哪些
        """
        global abstractMap
        try:
            title = abstractMap['nm']
        except:
            return None
        query = f"match(m:MovieItem)-[:Movie2Person]->(n:PersonItem) where m.title='{title}' return n.name"
        answer = self.g.run(query).data()
        res = title
        res += "电影的演员有"
        if answer:
            for i in range(len(answer)-1):
                res += answer[i].get('n.name')
                res += "、"
            res += answer[len(answer)-1].get('n.name')
            res += "。"
            return res
        return None
    def getMovieTypes(self,result):
        """3 对应问题模板3 == nm(电影) 类型

        :param result: 电影类型
        :return: 返回电影的类型
        """
        global abstractMap
        try:
            title = abstractMap['nm']
        except:
            return None
        query = f"match(m:MovieItem)-[Movie2Genre]->(n:GenreItem) where m.title='{title}' return n.gname"
        answer = self.g.run(query).data()
        if answer:
            return answer[0].get('n.gname')
        return None
    def getMovieRating(self, result):
        """4 对应问题模板4 == nm(电影) 评分

        :param result: 电影标题
        :return: 返回电影的评分
        """
        # **nm 评分 == 电影评分*/
        global abstractMap
        try:
            title = abstractMap['nm']
        except:
            return None
        query = f"match(n:MovieItem) where n.title='{title}' return n.rating"
        answer = self.g.run(query).data()
        if answer:
            return answer[0].get('n.rating')
        return None    
    def getActorMoviesType(self, result):
        """5 对应问题模板5 == nr(演员) 电影类型

        :param result: 电影标题
        :return: 返回电影的评分
        """
        global abstractMap
        try:
            name = abstractMap['nr']
        except:
            return None
#         tmp_mv="" 不会被后面替换
        query = f"match(n:PersonItem)-[:Person2Movie]->(m:MovieItem) where n.name='{name}' return m.title"
        answer = self.g.run(query).data()
        res = name
        res += "演过"
        l = []
        if answer:
            for i in range(len(answer)-1):
                tmp_mv = answer[i].get('m.title')
                query2 = f"match(m:MovieItem)-[:Movie2Genre]->(n:GenreItem) where m.title=\""+tmp_mv+"\" return n.gname"
                tmp_res = self.g.run(query2).data()
                if tmp_res:
                    tmp_res = tmp_res[0].get('n.gname')
                if tmp_res and tmp_res not in l:
                    l.append(tmp_res)
                    res += tmp_res
                    res += "、"
            tmp_mv = answer[len(answer)-1].get('m.title')
            tmp_res = self.g.run(query2).data()
            if tmp_res:
                tmp_res = tmp_res[0].get('n.gname')
            if tmp_res and tmp_res not in l:
                l.append(tmp_res)
                res+= tmp_res
            res+="类型的电影。"
            return res
        return None
    def getMovieReleaseDate(self, result):
        """6 对应问题模板6 == nm(电影) 上映时间

        :param result: 电影标题
        :return: 返回电影的评分
        """
        global abstractMap
        try:
            title = abstractMap['nm']
        except:
            return None
        query = f"match(m:MovieItem) where m.title=\""+title+"\" return m.releasedate"
        answer = self.g.run(query).data()
        if answer:
            return answer[0].get('m.releasedate')
        return None
    def getSameMovies(self, result):
        """7 对应问题模板7 == nr(a演员) ntr(b演员) 合作工作

        :return: 返回合作工作
        """
        global abstractMap
        try:
            nr = abstractMap['nr']
            ntr = abstractMap['ntr']
        except:
            return None
        query = f"match(n:PersonItem)-[Person2Movie]->(m:MovieItem) where n.name='{nr}' return m.title"
        query2 = f"match(n:PersonItem)-[Person2Movie]->(m:MovieItem) where n.name='{ntr}' return m.title"
        answer1 = self.g.run(query).data()
        answer2 = self.g.run(query2).data()
        if answer1 and answer2:
            # 两个list取交集
            intersection_d = [val for val in answer1 if val in answer2]
            if intersection_d:
                res = nr
                res += "和"
                res += ntr
                res += "合作了"
                for i in range(len(intersection_d)-1):
                    res += intersection_d[i].get('m.title')
                    res +='、'
                res += intersection_d[len(intersection_d)-1].get('m.title')
                res +="。"
            else:
                res = nr+"和"+ntr+"没合作过电影作品。"
            return res
        return None
    def getMoviesCount(self, result):
        """8 对应问题模板8 == nr(演员) 出演过多少部电影

        :return: 出演过多少部电影
        """
        global abstractMap
        try:
            nr = abstractMap['nr']
        except:
            return None
        query = f"match(n:PersonItem)-[:Person2Movie]->(m:MovieItem) where n.name='{nr}' return m.title"
        answer = self.g.run(query).data()
        if answer:
            res = nr+"出演过"+ str(len(answer)) +"部电影。"
            return res
        

In [22]:
vocabulary = load_vocabulary()
#cv=TfidfVectorizer(binary=False,decode_error='ignore',stopwords='english')
cv=TfidfVectorizer(binary=False,decode_error='ignore')
vec=cv.fit_transform(vocabulary)
arr=vec.toarray() #文本特征值矩阵向量arr

In [23]:
cv

TfidfVectorizer(decode_error='ignore')

In [24]:
arr.shape

(51, 32)

In [ ]:
a = np.array(list(map(lambda x: dicts[x.strip()], corpus_tags))) #标签矩阵向量a

In [77]:
if __name__ == '__main__':
    questions_pattern = load_question_templates()
    vocabulary = load_vocabulary()
    
    X, Y = load_data()
    # 朴素贝叶斯
    clf = MultinomialNB()
    clf.fit(X, Y)

    question_service = QuestionService()
    # 模板1的测试
#     print(question_service.answer('周星驰有什么作品'))
#     print(question_service.answer('Chen Dao-Ming 有什么作品'))
    # 模板2的测试
#     print(question_service.answer("Yat luk che有哪些人出演"))
    # 模板3的测试
#     print(question_service.answer("Yat luk che是什么类型的电影"))
#     print(question_service.answer("英雄是什么风格的电影"))
    # 模板4的测试
#     print(question_service.answer('Yat luk che这部电影的评分是多少'))
    # 模板5的测试
#     print(question_service.answer('李连杰演过哪些类型的电影'))
    # 模板6的测试
    print(question_service.answer('Yat luk che 电影的上映时间是什么时候'))
    # 模板7的测试
#     print(question_service.answer('章子怡和李连杰合作过哪些电影'))
    # 模板8的测试
#     print(question_service.answer('章子怡出演过多少部电影'))
#     abstractMap = {}

原始句子：Yat luk che 电影的上映时间是什么时候
========HanLP开始分词========
Yat luk che/nm
 /w
电影/n
的/ude1
上映/v
时间/n
是/vshi
什么/ry
时候/n
========HanLP分词结束========
句子抽象化结果：nm   电影 的 上映 时间 是 什么 时候 
the model index is 6
句子套用模板结果：nm 什么时候首播
原始句子替换成系统可识别的结果：Yat luk che 什么时候首播
2002/9/19


In [98]:
question_service = QuestionService()


answer = question_service.g.run("MATCH (n:MovieItem{title:\"英雄\"})-[rel:Movie2Person]-(entity2)  RETURN n.title,rel.type,entity2.name").data()



In [99]:
len(answer)

6

In [101]:
answer[0]

{'n.title': '英雄', 'rel.type': '演员', 'entity2.name': '梁朝伟'}

In [29]:
# # 朴素贝叶斯
# clf = MultinomialNB()
# clf.fit(X, Y)

MultinomialNB()

In [30]:
# question_service = QuestionService()

In [30]:
lt = [{"entity1": {"rating": "7.199999809", "mid": "146", "releasedate": "2000/10/1", "title": "卧虎藏龙", "introduction": "一代大侠李慕白有退出江湖之意，托付红颜知己俞秀莲将自己的青冥剑带到京城，作为礼物送给贝勒爷收藏。这把有四百年历史的古剑伤人无数，李慕白希望如此重大决断能够表明他离开江湖恩怨的决心。谁知当天夜里宝剑就被人盗走，俞秀莲上前阻拦与盗剑人交手，但最后盗剑人在同伙的救助下逃走。有人看见一个蒙面人消失在九门提督玉大人府内，俞秀莲也认为玉大人难逃干系。九门提督主管京城治安，玉大人刚从新疆调来赴任，贝勒爷即不相信玉大人与此有关，也不能轻举妄动以免影响大局。 　　俞秀莲为了不将事情复杂化一直在暗中查访宝剑下落，也大约猜出是玉府小姐玉蛟龙一时意气所为。俞秀莲对前来京城的李慕白隐瞒消息，只想用旁敲侧击的方法迫使玉蛟龙归还宝剑，免伤和气。不过俞秀莲的良苦用心落空，蒙面人真的归还宝剑时，不可避免地跟李慕白有了一次正面的交锋。而李慕白又发现了害死师傅的碧眼狐狸的踪迹，此时李慕白更是欲罢不能。 　　玉蛟龙自幼被隐匿于玉府的碧眼狐狸暗中收为弟子，并从秘籍中习得武当派上乘武功，早已青出于蓝。在新疆之时，玉蛟龙就瞒着父亲与当地大盗“半天云”罗小虎情定终身，如今身在北京，父亲又要她嫁人，玉蛟龙一时兴起冲出家门浪迹江湖。 　　任性傲气的玉蛟龙心中凄苦无处发泄，在江湖上使性任气，俨然是个小魔星。俞秀莲和李慕白爱惜玉蛟龙人才难得，苦心引导，总是无效。在最后和碧眼狐狸的交手之中，李慕白为救玉蛟龙身中毒针而死。玉蛟龙在俞秀莲的指点下来到武当山，却无法面对罗小虎，在和罗小虎一夕缠绵之后，投身万丈绝壑。"}, "rel": {"type": "演员"}, "entity2": {"birthplace": "Shanghai, China", "name": "Cheng Pei-Pei", "birth": "1946/12/4", "pid": "1624"}, "": ""}, {"entity1": {"rating": "7.199999809", "mid": "146", "releasedate": "2000/10/1", "title": "卧虎藏龙", "introduction": "一代大侠李慕白有退出江湖之意，托付红颜知己俞秀莲将自己的青冥剑带到京城，作为礼物送给贝勒爷收藏。这把有四百年历史的古剑伤人无数，李慕白希望如此重大决断能够表明他离开江湖恩怨的决心。谁知当天夜里宝剑就被人盗走，俞秀莲上前阻拦与盗剑人交手，但最后盗剑人在同伙的救助下逃走。有人看见一个蒙面人消失在九门提督玉大人府内，俞秀莲也认为玉大人难逃干系。九门提督主管京城治安，玉大人刚从新疆调来赴任，贝勒爷即不相信玉大人与此有关，也不能轻举妄动以免影响大局。 　　俞秀莲为了不将事情复杂化一直在暗中查访宝剑下落，也大约猜出是玉府小姐玉蛟龙一时意气所为。俞秀莲对前来京城的李慕白隐瞒消息，只想用旁敲侧击的方法迫使玉蛟龙归还宝剑，免伤和气。不过俞秀莲的良苦用心落空，蒙面人真的归还宝剑时，不可避免地跟李慕白有了一次正面的交锋。而李慕白又发现了害死师傅的碧眼狐狸的踪迹，此时李慕白更是欲罢不能。 　　玉蛟龙自幼被隐匿于玉府的碧眼狐狸暗中收为弟子，并从秘籍中习得武当派上乘武功，早已青出于蓝。在新疆之时，玉蛟龙就瞒着父亲与当地大盗“半天云”罗小虎情定终身，如今身在北京，父亲又要她嫁人，玉蛟龙一时兴起冲出家门浪迹江湖。 　　任性傲气的玉蛟龙心中凄苦无处发泄，在江湖上使性任气，俨然是个小魔星。俞秀莲和李慕白爱惜玉蛟龙人才难得，苦心引导，总是无效。在最后和碧眼狐狸的交手之中，李慕白为救玉蛟龙身中毒针而死。玉蛟龙在俞秀莲的指点下来到武当山，却无法面对罗小虎，在和罗小虎一夕缠绵之后，投身万丈绝壑。"}, "rel": {"type": "演员"}, "entity2": {"birthplace": "Lamma Island, Hong Kong", "name": "周润发", "birth": "1955/5/18", "pid": "1619", "biography": "周润发，广东宝安人，1955年5月18日生于香港南丫岛，为新界原居民，著名电影和电视演员，曾经三度获得香港电影金像奖最佳男主角奖，另外也拿到两次台湾电影金马奖最佳男主角头衔。 1980年代与成龙一并成为香港电影市场的票房保证，并创造了多个脍炙人口的经典角色：许文强、小马哥、赌神等多不胜数。1990年代开始后，与成龙、周星驰因为电影票房成绩出色，共同被媒体形容为“双周一成”。他于1995年赴美国好莱坞发展，在拍摄多部电影后获得一定程度上的成功。"}, "": ""}, {"entity1": {"rating": "7.199999809", "mid": "146", "releasedate": "2000/10/1", "title": "卧虎藏龙", "introduction": "一代大侠李慕白有退出江湖之意，托付红颜知己俞秀莲将自己的青冥剑带到京城，作为礼物送给贝勒爷收藏。这把有四百年历史的古剑伤人无数，李慕白希望如此重大决断能够表明他离开江湖恩怨的决心。谁知当天夜里宝剑就被人盗走，俞秀莲上前阻拦与盗剑人交手，但最后盗剑人在同伙的救助下逃走。有人看见一个蒙面人消失在九门提督玉大人府内，俞秀莲也认为玉大人难逃干系。九门提督主管京城治安，玉大人刚从新疆调来赴任，贝勒爷即不相信玉大人与此有关，也不能轻举妄动以免影响大局。 　　俞秀莲为了不将事情复杂化一直在暗中查访宝剑下落，也大约猜出是玉府小姐玉蛟龙一时意气所为。俞秀莲对前来京城的李慕白隐瞒消息，只想用旁敲侧击的方法迫使玉蛟龙归还宝剑，免伤和气。不过俞秀莲的良苦用心落空，蒙面人真的归还宝剑时，不可避免地跟李慕白有了一次正面的交锋。而李慕白又发现了害死师傅的碧眼狐狸的踪迹，此时李慕白更是欲罢不能。 　　玉蛟龙自幼被隐匿于玉府的碧眼狐狸暗中收为弟子，并从秘籍中习得武当派上乘武功，早已青出于蓝。在新疆之时，玉蛟龙就瞒着父亲与当地大盗“半天云”罗小虎情定终身，如今身在北京，父亲又要她嫁人，玉蛟龙一时兴起冲出家门浪迹江湖。 　　任性傲气的玉蛟龙心中凄苦无处发泄，在江湖上使性任气，俨然是个小魔星。俞秀莲和李慕白爱惜玉蛟龙人才难得，苦心引导，总是无效。在最后和碧眼狐狸的交手之中，李慕白为救玉蛟龙身中毒针而死。玉蛟龙在俞秀莲的指点下来到武当山，却无法面对罗小虎，在和罗小虎一夕缠绵之后，投身万丈绝壑。"}, "rel": {"type": "演员"}, "entity2": {"birthplace": "Beijing , China", "name": "章子怡", "birth": "1979/2/9", "pid": "1339"}, "": ""}]

In [ ]:
def getEntityRelationbyEntity(self, value):

    answer = self.graph.run("MATCH (entity1) - [rel] -> (entity2)  WHERE entity1.name = \"" + str(
                value) + "\" RETURN entity1,rel,entity2").data()
    ##  要用if not answer , 而不是判断是不是none，因为没有答案就是【】
    if not answer:
        # 关系查询中，如果是movie的查询，只返回电影的演员
        answer = self.graph.run("MATCH (entity1) - [rel:Movie2Person] -> (entity2)  WHERE entity1.title = \"" + str(
            value) + "\" RETURN entity1,rel,entity2").data()
    if not answer:
        answer = self.graph.run("MATCH (entity1) - [rel] -> (entity2)  WHERE entity1.gname = \"" + str(
            value) + "\" RETURN entity1,rel,entity2").data()
    return answer

In [112]:
lt[0]["entity2"]

{'birthplace': 'Shanghai, China',
 'name': 'Cheng Pei-Pei',
 'birth': '1946/12/4',
 'pid': '1624'}

In [109]:
if "title" in lt[0]["entity2"]:
    print(True)